# Prática Supervisionada

In [1]:
import pandas as pd
from sklearn import model_selection
from sklearn import preprocessing

## Carregue o dataset wine.data

# Dividir os dados entre treino e teste de forma estratificada

# Provar a estratificação dos dados com relação aos labels

# Usar Grid Search com validação cruzada 5-Fold e diferentes parâmetros para diversos modelos e encontrar o melhor deles. Aplicar Standardization nos dados.